In [ ]:
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.model_selection import ParameterSampler
from scipy.stats import loguniform
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

In [ ]:
#Do radom search for hyperparameter tuning for a MLP model on the full balanced dataset
data = pd.read_csv('C:/Users/Rasmu/OneDrive/DTU/Kunstig intelligens og Data/7. semestre/Bachelor/Machine-Learning-For-Network-Traffic-Analysis/Data/combined_data/final_data_balanced.csv')

X = data.drop(['label', 'detailed_label'], axis=1)
y = data['label']

#check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#Define the parameter grid
param_grid = {
    'units_layer0': [128, 256, 512],
    'units_layer1': [32, 64, 128],
    'units_layer2': [8, 16, 32],
    'units_layer3': [2, 4, 8],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30, 40, 50],
    'dropout_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'learning_rate': loguniform(1e-4, 1e-1),
}

n_iter = 60 #Times to try different hyperparameters
random_search = list(ParameterSampler(param_grid, n_iter=n_iter, random_state=42)) #Define the 'random search'

n_splits = 5 #Number of K-folds
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42) #Create the K-folds

#MLP function
def create_mlp(parameter, input_shape):
    model = Sequential()
    model.add(keras.Input(shape=(input_shape,)))
    model.add(Dense(parameter['units_layer0'], activation='relu'))
    model.add(Dropout(parameter['dropout_rate']))
    model.add(Dense(parameter['units_layer1'], activation='relu'))
    model.add(Dropout(parameter['dropout_rate']))
    model.add(Dense(parameter['units_layer2'], activation='relu'))
    model.add(Dropout(parameter['dropout_rate']))
    model.add(Dense(parameter['units_layer3'], activation='relu'))
    model.add(Dropout(parameter['dropout_rate']))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=parameter['learning_rate'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

#Here the code runs the random search
results = []
for i, parameter in enumerate(random_search):
    print(f"Trial {i+1}/{n_iter}: {parameter}")
    fold_accuracies = []
    fold_losses = []
    
    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        model = create_mlp(parameter, input_shape=X.shape[1])
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        #Train by using GPU, if available otherwise use CPU
        with tf.device('/GPU:0'):
            model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=parameter['epochs'], batch_size=parameter['batch_size'], callbacks=[early_stopping], verbose=0)

        val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
        fold_accuracies.append(val_accuracy)
        fold_losses.append(val_loss)
    
    #Average results across folds, to ensure that the chosen model is not overfitting
    avg_accuracy = sum(fold_accuracies) / n_splits
    avg_loss = sum(fold_losses) / n_splits
    results.append({'parameters': parameter, 'avg_loss': avg_loss, 'avg_accuracy': avg_accuracy})

#Print best configuration
best_model = max(results, key=lambda x: x['avg_accuracy'])
print("Best parameters:", best_model['params'])
print("Validation Accuracy:", best_model['avg_accuracy'])


In [ ]:
#Do radom search for hyperparameter tuning for a MLP model on the selected features
data_path = 'C:/Users/Rasmu/OneDrive/DTU/Kunstig intelligens og Data/7. semestre/Bachelor/Machine-Learning-For-Network-Traffic-Analysis/Data/combined_data/final_data_balanced.csv'
data = pd.read_csv(data_path)
#Only use the selected features
X = data[['conn_state_RSTOS0', 'orig_pkts', 'resp_bytes', 'resp_pkts', 'orig_bytes']]
y= data['label']

#check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#Define the parameter grid
param_grid = {
    'units_layer0': [128, 256, 512],
    'units_layer1': [32, 64, 128],
    'units_layer2': [8, 16, 32],
    'units_layer3': [4, 8],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30, 40, 50],
    'dropout_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'learning_rate': loguniform(1e-4, 1e-1),
}

n_iter = 60 #Times to try different hyperparameters
random_search = list(ParameterSampler(param_grid, n_iter=n_iter, random_state=42)) #Define teh 'random search'


n_splits = 5 #Number of K-folds
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42) #Create the K-folds

#MLP function
def create_mlp(parameter, input_shape):
    model = Sequential()
    model.add(keras.Input(shape=(input_shape,)))
    model.add(Dense(parameter['units_layer0'], activation='relu'))
    model.add(Dropout(parameter['dropout_rate']))
    model.add(Dense(parameter['units_layer1'], activation='relu'))
    model.add(Dropout(parameter['dropout_rate']))
    model.add(Dense(parameter['units_layer2'], activation='relu'))
    model.add(Dropout(parameter['dropout_rate']))
    model.add(Dense(parameter['units_layer3'], activation='relu'))
    model.add(Dropout(parameter['dropout_rate']))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=parameter['learning_rate'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

#Here the code runs the random search
results = []
for i, parameter in enumerate(random_search):
    print(f"Trial {i+1}/{n_iter}: {parameter}")
    fold_accuracies = []
    fold_losses = []
    
    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        model = create_mlp(parameter, input_shape=X.shape[1])
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        #Train by using GPU, if available otherwise use CPU
        with tf.device('/GPU:0'):
            model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=parameter['epochs'], batch_size=parameter['batch_size'], callbacks=[early_stopping], verbose=0)

        val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
        fold_accuracies.append(val_accuracy)
        fold_losses.append(val_loss)
    
    #Average results across folds, to ensure that the chosen model is not overfitting
    avg_accuracy = sum(fold_accuracies) / n_splits
    avg_loss = sum(fold_losses) / n_splits
    results.append({'parameters': parameter, 'avg_loss': avg_loss, 'avg_accuracy': avg_accuracy})

#Print best configuration
best_model = max(results, key=lambda x: x['avg_accuracy'])
print("Best parameters:", best_model['params'])
print("Validation Accuracy:", best_model['avg_accuracy'])